In [7]:
%%capture captured_output
%pip install PyPDF2 openai lancedb 

import os
import math
import PyPDF2
import lancedb
import pyarrow as pa
from openai import OpenAI
from langchain.text_splitter import TokenTextSplitter


OPENAI_API_KEY = "ENTER_YOUR_API_KEY"
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    print(f"Processing {file_path}...")
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        print(f"- Found {len(reader.pages)} pages")
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text() + "\n"
            if (page_num + 1) % 20 == 0:  # Progress every 20 pages
                print(f"- Processed {page_num + 1} pages")
    return text

In [9]:
def chunk_text(text, chunk_size=4000, overlap=256):
    """Split text into overlapping chunks."""
    text_splitter = TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        encoding_name="cl100k_base"
    )
    chunks = text_splitter.split_text(text)
    print(f"Created {len(chunks)} chunks")
    return chunks

In [10]:
def extract_company_from_filename(filename):
    """Extract company code from filename."""
    company = filename.replace('.pdf', '').upper()
    company_mapping = {
        'ALL': 'ALL', 
        'CHUBB': 'CHUBB',
        'PGR': 'PGR',
        'TRV': 'TRV'
    }
    return company_mapping.get(company, company)

In [11]:
def get_embedding(texts):
    """Get embeddings for text using OpenAI's API."""
    if isinstance(texts, str):
        texts = [texts]
        single_input = True
    else:
        single_input = False
    
    batch_size = 100
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_end = min(i + batch_size, len(texts))
        print(f"Processing embeddings for batch {i//batch_size + 1}, documents {i} to {batch_end-1}")
        
        response = client.embeddings.create(
            input=batch,
            model="text-embedding-ada-002"
        )
        batch_embeddings = [data.embedding for data in response.data]
        embeddings.extend(batch_embeddings)
    
    return embeddings[0] if single_input else embeddings

In [12]:
def prepare_database():
    # Process all PDFs first
    pdf_files = ["all.pdf", "chubb.pdf", "pgr.pdf", "trv.pdf"]
    documents = []
    
    for pdf_file in pdf_files:
        if not os.path.exists(pdf_file):
            print(f"File not found: {pdf_file}")
            continue
            
        company = pdf_file.replace('.pdf', '').upper()
        text = extract_text_from_pdf(pdf_file)
        chunks = chunk_text(text)
        print(f"Processing {pdf_file}: {len(chunks)} chunks")
        
        for idx, chunk in enumerate(chunks):
            documents.append({
                "id": f"{company}_{idx}",
                "source": pdf_file,
                "text": chunk,
                "company": company,
            })

    print(f"Total chunks: {len(documents)}")
    
    # Get embeddings for all texts at once
    texts = [doc["text"] for doc in documents]
    embeddings = get_embedding(texts)
    
    # Add embeddings to documents
    for doc, embedding in zip(documents, embeddings):
        doc["vector"] = embedding

    # Create table with explicit schema definition for fixed-size list
    db = lancedb.connect("lancedb_insurance_competition")
    table_name = "documents"
    
    try:
        db.drop_table(table_name)
    except:
        pass

    # Define schema with explicit list_size for vector
    schema = pa.schema([
        ("id", pa.string()),
        ("source", pa.string()),
        ("text", pa.string()),
        ("vector", pa.list_(pa.float32(), 1536)),  # Fixed size list with 1536 dimensions
        ("company", pa.string()),
    ])

    # Create Arrow Table with correct schema
    arrays = [
        pa.array([doc["id"] for doc in documents]),
        pa.array([doc["source"] for doc in documents]),
        pa.array([doc["text"] for doc in documents]),
        pa.FixedSizeListArray.from_arrays(
            pa.array([v for doc in documents for v in doc["vector"]], type=pa.float32()),
            1536
        ),
        pa.array([doc["company"] for doc in documents])
    ]
    arrow_table = pa.Table.from_arrays(arrays, schema=schema)

    # Create LanceDB table from Arrow Table
    table = db.create_table(table_name, data=arrow_table, mode="overwrite")
    print(f"Created database with {len(documents)} documents")

if __name__ == "__main__":
    prepare_database()

Processing all.pdf...
- Found 251 pages
- Processed 20 pages
- Processed 40 pages
- Processed 60 pages
- Processed 80 pages
- Processed 100 pages
- Processed 120 pages
- Processed 140 pages
- Processed 160 pages
- Processed 180 pages
- Processed 200 pages
- Processed 220 pages
- Processed 240 pages
Created 50 chunks
Processing all.pdf: 50 chunks
Processing chubb.pdf...
- Found 356 pages
- Processed 20 pages
- Processed 40 pages
- Processed 60 pages
- Processed 80 pages
- Processed 100 pages
- Processed 120 pages
- Processed 140 pages
- Processed 160 pages
- Processed 180 pages
- Processed 200 pages
- Processed 220 pages
- Processed 240 pages
- Processed 260 pages
- Processed 280 pages
- Processed 300 pages
- Processed 320 pages
- Processed 340 pages
Created 53 chunks
Processing chubb.pdf: 53 chunks
Processing pgr.pdf...
- Found 79 pages
- Processed 20 pages
- Processed 40 pages
- Processed 60 pages
Created 12 chunks
Processing pgr.pdf: 12 chunks
Processing trv.pdf...
- Found 244 pages


[2025-02-07T05:23:17Z WARN  lance::dataset::write::insert] No existing dataset at /Users/andrey/Dev/zesty/lancedb_insurance_competition/documents.lance, it will be created


In [13]:
import lancedb
import pandas as pd

# --- 1. Connect to the database ---
db = lancedb.connect("lancedb_insurance_competition")

# --- 2. Open the table ---
table = db.open_table("documents")

# --- 3. Print the table schema (structure) ---
print("Table Schema:")
print(table.schema)

# --- 4. Get the total row count reported by the table ---
reported_count = table.count_rows()
print("\nTotal row count reported by DB:", reported_count)

# --- 5. Retrieve all rows using the search interface with a high limit ---
# Using search(None) tells LanceDB you want all rows.
full_data = table.search(None).limit(1000).to_pandas()
retrieved_count = len(full_data)
print("\nRetrieved row count:", retrieved_count)

if retrieved_count == reported_count:
    print("\nSuccess: All data is present in the table.")
else:
    print(f"\nWarning: Expected {reported_count} rows but retrieved {retrieved_count} rows.")

# --- 6. Show how many documents exist for each company ---
print("\nDocuments per company:")
company_counts = full_data['company'].value_counts()
print(company_counts)


Table Schema:
id: string
source: string
text: string
vector: fixed_size_list<item: float>[1536]
  child 0, item: float
company: string

Total row count reported by DB: 161

Retrieved row count: 161

Success: All data is present in the table.

Documents per company:
company
CHUBB    53
ALL      50
TRV      46
PGR      12
Name: count, dtype: int64
